<a href="https://colab.research.google.com/github/jaimehdzgt/superstore_project/blob/main/SuperStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==== CONFIG ====
REPO_URL   = "https://github.com/jaimehdzgt/superstore_project.git"
BRANCH     = "main"
REPO_NAME  = "superstore_project"
REPO_DIR   = f"/content/{REPO_NAME}"

# Si True, borra cualquier clon previo y vuelve a clonar limpio.
# Si False, sólo hace pull/actualiza sin duplicar.
CLEAN_RUN  = True

# Carpeta en tu Drive y patrón del archivo Excel:
DRIVE_FOLDER   = "/content/drive/MyDrive/SuperStore"
EXCEL_PATTERN  = "Sample - Superstore*.xlsx"   # tolera el espacio antes de .xlsx

print("Config OK:", REPO_URL, "branch:", BRANCH, "clean:", CLEAN_RUN)


import os, sys, glob, subprocess, shutil
from pathlib import Path

def run(cmd, check=True):
    print(">", cmd)
    rc = subprocess.call(cmd, shell=True)
    if check and rc != 0:
        raise RuntimeError(f"Falló: {cmd}")

# 1) Limpiar si se pidió CLEAN_RUN
if CLEAN_RUN and Path(REPO_DIR).exists():
    print(f"Eliminando clon previo: {REPO_DIR}")
    shutil.rmtree(REPO_DIR)

# 2) Clonar o actualizar sin duplicar
if not Path(REPO_DIR).exists():
    run(f"git clone -b {BRANCH} {REPO_URL} {REPO_DIR}")
else:
    # Actualiza el repo existente sin crear carpetas extra
    run(f"git -C {REPO_DIR} fetch origin {BRANCH}")
    run(f"git -C {REPO_DIR} reset --hard origin/{BRANCH}")
    run(f"git -C {REPO_DIR} clean -fdx")  # borra archivos sin trackear dentro del repo (no tu Drive)

# 3) Entrar a notebooks
%cd {REPO_DIR}/notebooks
!ls -la

# 4) Habilitar imports (src/) sólo una vez
if ".." not in sys.path:
    sys.path.append("..")

# 5) Instalar dependencias (idempotente; pip ignora lo ya instalado)
!pip install -q -r ../requirements.txt
print("Entorno listo.")


# Montar Drive sólo si no está montado
from google.colab import drive, files
import os
if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")

from pathlib import Path
import pandas as pd

# Buscar el Excel por patrón dentro de tu carpeta 'SuperStore'
base = Path(DRIVE_FOLDER)
assert base.exists(), f"No existe la carpeta de Drive: {base}"

candidatos = list(base.glob(EXCEL_PATTERN))
print("Candidatos encontrados:", candidatos)

if not candidatos:
    raise FileNotFoundError(
        f"No se encontró ningún Excel con patrón '{EXCEL_PATTERN}' en {base}.\n"
        "Revisa el nombre del archivo o renómbralo en Drive."
    )

# Toma el primero (ajusta índice si tuvieras varios)
excel_path = str(candidatos[0])
print("excel_path =", excel_path)

# Verificación de lectura (usa openpyxl)
df_head = pd.read_excel(excel_path, engine="openpyxl", nrows=5)
df_head


Config OK: https://github.com/jaimehdzgt/superstore_project.git branch: main clean: True
Eliminando clon previo: /content/superstore_project
> git clone -b main https://github.com/jaimehdzgt/superstore_project.git /content/superstore_project
/content/superstore_project/notebooks
total 28
drwxr-xr-x 2 root root 4096 Sep 23 19:41 .
drwxr-xr-x 6 root root 4096 Sep 23 19:41 ..
-rw-r--r-- 1 root root 9377 Sep 23 19:41 01_EDA_Superstore.ipynb
-rw-r--r-- 1 root root 5482 Sep 23 19:41 02_Modeling_Superstore.ipynb
Entorno listo.
Candidatos encontrados: [PosixPath('/content/drive/MyDrive/SuperStore/Sample - Superstore .xlsx')]
excel_path = /content/drive/MyDrive/SuperStore/Sample - Superstore .xlsx


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2025-11-08,2025-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2025-11-08,2025-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2025-06-12,2025-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2023-10-11,2023-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2023-10-11,2023-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [2]:
# %% [markdown]
# # 📌 EDA — Sample Superstore (Google Drive)
# - Monta Drive y encuentra automáticamente el Excel en: **/content/drive/MyDrive/SuperStore/**
# - Carga el dataset y realiza un EDA básico
# - Cuenta valores vacíos (totales y %) y duplicados
# - Exporta reporte de nulos y descriptivos a CSV (compatibles con cualquier pandas)

# %%
# ===== 0) Setup & Drive =====
from google.colab import drive
drive.mount('/content/drive')  # autoriza acceso a tu Drive

from pathlib import Path
from IPython.display import display
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Carpeta donde está tu archivo en Drive
BASE_DIR = Path('/content/drive/MyDrive/SuperStore')

# Patrón flexible: acepta "Sample - Superstore.xlsx" y también
# "Sample - Superstore .xlsx" (con espacio antes de .xlsx)
pattern = 'Sample - Superstore*.xlsx'

# Busca el archivo
candidatos = sorted(BASE_DIR.glob(pattern))
if not candidatos:
    raise FileNotFoundError(
        f"No se encontró ningún Excel con patrón '{pattern}' en {BASE_DIR}.\n"
        "Verifica la carpeta o renombra el archivo."
    )

# Toma el primero (si hay más, ajusta el índice)
excel_path = str(candidatos[0])
print("✅ Excel encontrado en:", excel_path)

# ===== 1) Carga =====
# Si falta openpyxl:  !pip install openpyxl
df = pd.read_excel(excel_path, engine="openpyxl")
print("Shape:", df.shape, "| Columnas:", len(df.columns))
display(df.head(3))

# ===== 2) Normalización ligera =====
df = df.copy()
df.columns = [c.strip().replace(" ", "_").replace("-", "_") for c in df.columns]

# Tipificar columnas con "date" en el nombre
for c in df.columns:
    if "date" in c.lower():
        try:
            df[c] = pd.to_datetime(df[c])
        except Exception:
            pass

# ===== 3) Vista rápida =====
print("\n🔹 Info del DataFrame")
df.info()

# ===== 4) Calidad de datos: vacíos/duplicados =====
nulls = df.isna().sum().to_frame("missing")
nulls["pct"] = (nulls["missing"] / len(df)).round(4)
nulls = nulls.sort_values("missing", ascending=False)

print("\n🔸 Nulos por columna (top 25):")
display(nulls.head(25))

total_missing_cells = int(nulls["missing"].sum())
total_cells = int(df.size)
print(f"\nTotal celdas faltantes: {total_missing_cells:,} de {total_cells:,} "
      f"({(total_missing_cells/total_cells):.2%})")

dups = df.duplicated().sum()
print(f"🔸 Filas duplicadas: {dups}")

# %% [markdown]
# ## 4.1) Actualizar Ship_Date = Order_Date + 5 días (sin perder el reporte previo de nulos)

# %%
# Nota: tras la normalización, las columnas se llaman 'Order_Date' y 'Ship_Date'
if "Order_Date" in df.columns:
    # Asegura que Ship_Date exista y sea datetime sin romper el conteo de nulos previo
    if "Ship_Date" not in df.columns:
        df["Ship_Date"] = pd.NaT

    # Forzar tipo datetime (si hay valores no convertibles, quedan como NaT)
    df["Ship_Date"] = pd.to_datetime(df["Ship_Date"], errors="coerce")
    df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors="coerce")

    # Guarda cuántos nulos había antes (para comparar)
    before_nulls_ship = int(df["Ship_Date"].isna().sum())

    # Actualiza Ship_Date solo donde exista Order_Date
    mask = df["Order_Date"].notna()
    df.loc[mask, "Ship_Date"] = df.loc[mask, "Order_Date"] + pd.to_timedelta(5, unit="D")

    after_nulls_ship = int(df["Ship_Date"].isna().sum())
    print(f"Ship_Date nulos — antes: {before_nulls_ship} | después: {after_nulls_ship} (post-actualización +5d)")
else:
    print("⚠️ No se encontró la columna 'Order_Date'; no se puede calcular Ship_Date = Order_Date + 5 días.")


# ===== 5) Descriptivos =====
desc_num = df.select_dtypes(include=np.number).describe().T
print("\n📊 Descriptivos (numéricos):")
display(desc_num.head(20))



# ===== 6) Outliers (IQR) =====
def iqr_flags(s, k=1.5):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - k*iqr, q3 + k*iqr
    return (s < low) | (s > high), dict(q1=q1, q3=q3, iqr=iqr, low=low, high=high)

for col in [c for c in ["Sales","Profit","Quantity","Discount"] if c in df.columns]:
    s = df[col].dropna()
    flags, stats = iqr_flags(s)
    print(f"Outliers {col}: {int(flags.sum())}/{s.size}  |  stats={{k: float(v) for k,v in stats.items()}}")

# ===== 7) Correlaciones (numéricas) =====
num = df.select_dtypes(include=np.number)
if num.shape[1] >= 2:
    corr = num.corr(numeric_only=True)
    print("\n🔗 Matriz de correlación (primeras 10 columnas):")
    display(corr.iloc[:10, :10])
else:
    print("\nNo hay suficientes columnas numéricas para correlación.")

# ===== 8) Guardados útiles (compatibles con cualquier pandas) =====
OUT_DIR = Path("/content/eda_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def describe_all_compat(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    'describe' combinado para numéricas, fechas y objetos.
    Evita usar 'datetime_is_numeric' para compatibilidad.
    """
    # Numéricas
    num_stats = dataframe.select_dtypes(include=np.number).describe().T
    if not num_stats.empty:
        num_stats["__type__"] = "numeric"

    # Datetime
    dt_cols = dataframe.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns
    if len(dt_cols):
        dt_stats = dataframe[dt_cols].agg(["min", "max", "nunique", "count"]).T
        dt_stats["__type__"] = "datetime"
    else:
        dt_stats = pd.DataFrame()

    # Objetos
    obj_stats = dataframe.select_dtypes(include="object").describe().T
    if not obj_stats.empty:
        obj_stats["__type__"] = "object"

    parts = [p for p in [num_stats, dt_stats, obj_stats] if not p.empty]
    out = pd.concat(parts, axis=0) if parts else pd.DataFrame()

    if not out.empty:
        cols = ["__type__"] + [c for c in out.columns if c != "__type__"]
        out = out[cols]
    return out

# Descriptivos completos a CSV
desc_all = describe_all_compat(df)
desc_all.to_csv(OUT_DIR / "describe_all.csv")
print("\n💾 Descriptivos guardados en:", (OUT_DIR / "describe_all.csv"))
display(desc_all.head(20))

# Reporte de nulos
nulls.to_csv(OUT_DIR / "missing_report.csv")
print("💾 Reporte de nulos guardado en:", (OUT_DIR / "missing_report.csv"))

# Muestra limpia a CSV (para compartir/subir a GitHub)
sample_csv = OUT_DIR / "superstore_sample_clean.csv"
df.sample(min(1000, len(df)), random_state=42).to_csv(sample_csv, index=False)
print("💾 Muestra guardada en:", sample_csv)

# ===== 9) Resumen =====
print("\nResumen:")
print("• excel_path:", excel_path)
print("• shape:", df.shape)
print("• nulos totales:", int(df.isna().sum().sum()))
print("• duplicados:", int(df.duplicated().sum()))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Excel encontrado en: /content/drive/MyDrive/SuperStore/Sample - Superstore .xlsx
Shape: (9994, 21) | Columnas: 21


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2025-11-08,2025-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2025-11-08,2025-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2025-06-12,2025-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714



🔹 Info del DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row_ID         9994 non-null   int64         
 1   Order_ID       9994 non-null   object        
 2   Order_Date     9994 non-null   datetime64[ns]
 3   Ship_Date      9888 non-null   datetime64[ns]
 4   Ship_Mode      9994 non-null   object        
 5   Customer_ID    9994 non-null   object        
 6   Customer_Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal_Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product_ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15 

,missing,pct
Ship_Date,106,0.0106
Row_ID,0,0.0000
Order_ID,0,0.0000
Order_Date,0,0.0000
Ship_Mode,0,0.0000
Customer_ID,0,0.0000
Customer_Name,0,0.0000
Segment,0,0.0000
Country,0,0.0000
City,0,0.0000



Total celdas faltantes: 106 de 209,874 (0.05%)
🔸 Filas duplicadas: 0
Ship_Date nulos — antes: 106 | después: 0 (post-actualización +5d)

📊 Descriptivos (numéricos):


...

Outliers Sales: 1167/9994  |  stats={k: float(v) for k,v in stats.items()}
Outliers Profit: 1881/9994  |  stats={k: float(v) for k,v in stats.items()}
Outliers Quantity: 170/9994  |  stats={k: float(v) for k,v in stats.items()}
Outliers Discount: 856/9994  |  stats={k: float(v) for k,v in stats.items()}

🔗 Matriz de correlación (primeras 10 columnas):


,Row_ID,Postal_Code,Sales,Quantity,Discount,Profit
Row_ID,1.000000,0.009671,-0.001359,-0.004016,0.013480,0.012497
Postal_Code,0.009671,1.000000,-0.023854,0.012761,0.058443,-0.029961
Sales,-0.001359,-0.023854,1.000000,0.200795,-0.028190,0.479064
Quantity,-0.004016,0.012761,0.200795,1.000000,0.008623,0.066253
Discount,0.013480,0.058443,-0.028190,0.008623,1.000000,-0.219487
Profit,0.012497,-0.029961,0.479064,0.066253,-0.219487,1.000000



💾 Descriptivos guardados en: /content/eda_outputs/describe_all.csv


,__type__,count,mean,std,min,25%,50%,75%,max,nunique,unique,top,freq
Row_ID,numeric,9994.0,4997.500000,2885.163629,1.0,2499.25000,4997.5000,7495.750,9994.0,NaN,NaN,NaN,NaN
Postal_Code,numeric,9994.0,55190.379428,32063.693350,1040.0,23223.00000,56430.5000,90008.000,99301.0,NaN,NaN,NaN,NaN
Sales,numeric,9994.0,229.858001,623.245101,0.444,17.28000,54.4900,209.940,22638.48,NaN,NaN,NaN,NaN
Quantity,numeric,9994.0,3.789574,2.225110,1.0,2.00000,3.0000,5.000,14.0,NaN,NaN,NaN,NaN
Discount,numeric,9994.0,0.156203,0.206452,0.0,0.00000,0.2000,0.200,0.8,NaN,NaN,NaN,NaN
Profit,numeric,9994.0,28.656896,234.260108,-6599.978,1.72875,8.6665,29.364,8399.976,NaN,NaN,NaN,NaN
Order_Date,datetime,9994,NaN,NaN,2022-01-03 00:00:00,NaN,NaN,NaN,2025-12-31 00:00:00,1236,NaN,NaN,NaN
Ship_Date,datetime,9994,NaN,NaN,2022-01-08 00:00:00,NaN,NaN,NaN,2026-01-05 00:00:00,1236,NaN,NaN,NaN
Order_ID,object,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5009,CA-2017-100111,14
Ship_Mode,object,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Standard Class,5968


💾 Reporte de nulos guardado en: /content/eda_outputs/missing_report.csv
💾 Muestra guardada en: /content/eda_outputs/superstore_sample_clean.csv

Resumen:
• excel_path: /content/drive/MyDrive/SuperStore/Sample - Superstore .xlsx
• shape: (9994, 21)
• nulos totales: 0
• duplicados: 0


In [3]:
# %% [markdown]
# # 📌 Insights & Conclusiones — Superstore (Auto-Resumen)
# Este bloque genera un **informe con bookmarks** a partir del DataFrame `df` ya cargado/limpiado
# en tu notebook (usa las columnas normalizadas: Sales, Profit, Segment, Region, State,
# Category, Sub_Category, Ship_Mode, Order_Date, Ship_Date).
#
# - Si existe `/content/eda_outputs/missing_report.csv`, mostrará los **nulos previos** de Ship_Date.
# - No modifica `df`; solo calcula y muestra hallazgos.
#
# **Secciones**
# - [1) Calidad de datos](#ins-calidad)
# - [2) Clientes & Segmentos](#ins-segmentos)
# - [3) Geografía](#ins-geo)
# - [4) Categorías & Productos](#ins-cat)
# - [5) Logística & Envíos](#ins-ship)
# - [6) Rentabilidad global](#ins-rentab)
# - [7) Próximos pasos](#ins-next)

# %%
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

def safe_exists(col):
    return isinstance(col, str) and (col in df.columns)

def top_item(series):
    if series.empty:
        return None, None
    vc = series.value_counts(dropna=False)
    return (vc.index[0], int(vc.iloc[0]))

def pct(n, d):
    return f"{(100*n/d):.1f}%" if d else "0.0%"

def read_ship_nulls_before():
    # Intenta leer el reporte previo de nulos del paso de EDA
    path = Path("/content/eda_outputs/missing_report.csv")
    if path.exists():
        try:
            mr = pd.read_csv(path, index_col=0)
            # intenta localizar por cualquier forma del nombre
            for key in ["Ship_Date", "Ship Date", "Ship_Date "]:
                if key in mr.index:
                    return int(mr.loc[key, "missing"])
        except Exception:
            pass
    return None

# ===== Cálculos robustos =====
rows = len(df)
sales_col    = "Sales"      if safe_exists("Sales") else None
profit_col   = "Profit"     if safe_exists("Profit") else None
segment_col  = "Segment"    if safe_exists("Segment") else None
region_col   = "Region"     if safe_exists("Region") else None
state_col    = "State"      if safe_exists("State") else None
cat_col      = "Category"   if safe_exists("Category") else None
subcat_col   = "Sub_Category" if safe_exists("Sub_Category") else None
shipmode_col = "Ship_Mode"  if safe_exists("Ship_Mode") else None
order_col    = "Order_Date" if safe_exists("Order_Date") else None
ship_col     = "Ship_Date"  if safe_exists("Ship_Date") else None

# 1) Calidad (Ship_Date nulos antes/después)
ship_nulls_before = read_ship_nulls_before()
ship_nulls_after  = int(df[ship_col].isna().sum()) if ship_col else None

# 2) Segmentos
seg_counts = df[segment_col].value_counts(dropna=False) if segment_col else pd.Series(dtype=int)
seg_profit = df.groupby(segment_col)[profit_col].sum().sort_values(ascending=False) if (segment_col and profit_col) else pd.Series(dtype=float)

# 3) Geografía (Region / State)
reg_profit = df.groupby(region_col)[profit_col].sum().sort_values(ascending=False) if (region_col and profit_col) else pd.Series(dtype=float)
reg_sales  = df.groupby(region_col)[sales_col].sum().sort_values(ascending=False)  if (region_col and sales_col)  else pd.Series(dtype=float)
reg_margin = (reg_profit / reg_sales).replace([np.inf, -np.inf], np.nan) if (not reg_profit.empty and not reg_sales.empty) else pd.Series(dtype=float)

state_profit = df.groupby(state_col)[profit_col].sum().sort_values(ascending=False) if (state_col and profit_col) else pd.Series(dtype=float)

# 4) Categorías
cat_sales  = df.groupby(cat_col)[sales_col].sum().sort_values(ascending=False)  if (cat_col and sales_col)  else pd.Series(dtype=float)
cat_profit = df.groupby(cat_col)[profit_col].sum().sort_values(ascending=False) if (cat_col and profit_col) else pd.Series(dtype=float)
sub_profit = df.groupby(subcat_col)[profit_col].sum().sort_values(ascending=False) if (subcat_col and profit_col) else pd.Series(dtype=float)

# 5) Envíos
ship_counts = df[shipmode_col].value_counts(dropna=False) if shipmode_col else pd.Series(dtype=int)
ship_profit = df.groupby(shipmode_col)[profit_col].sum().sort_values(ascending=False) if (shipmode_col and profit_col) else pd.Series(dtype=float)

# 6) Rentabilidad global
profitable_rate = None
global_margin = None
if profit_col and sales_col:
    profitable_rate = (df[profit_col] > 0).mean()
    # evitar división por cero
    total_sales = df[sales_col].sum()
    total_profit = df[profit_col].sum()
    global_margin = (total_profit / total_sales) if total_sales else np.nan

# ===== Hallazgos en texto =====
lines = []

# Encabezado + TOC
lines += [
"# 📌 Insights & Conclusiones — Superstore",
"- [1) Calidad de datos](#ins-calidad)",
"- [2) Clientes & Segmentos](#ins-segmentos)",
"- [3) Geografía](#ins-geo)",
"- [4) Categorías & Productos](#ins-cat)",
"- [5) Logística & Envíos](#ins-ship)",
"- [6) Rentabilidad global](#ins-rentab)",
"- [7) Próximos pasos](#ins-next)",
"",
"<a id='ins-calidad'></a>",
"### 1) Calidad de datos",
]

# Calidad
if ship_col:
    if ship_nulls_before is not None:
        lines.append(f"- **Ship_Date con nulos (antes):** **{ship_nulls_before}** filas (del reporte de nulos previo).")
    lines.append(f"- **Ship_Date con nulos (actual):** **{int(ship_nulls_after)}** filas.")
    if order_col:
        lines.append("- La regla **Ship_Date = Order_Date + 5 días** se aplicó donde hubo *Order_Date* disponible.")
else:
    lines.append("- No se encontró la columna **Ship_Date** en el DataFrame.")

# Segmentos
lines += ["", "<a id='ins-segmentos'></a>", "### 2) Clientes & Segmentos"]
if not seg_counts.empty:
    seg_top, seg_top_n = seg_counts.index[0], int(seg_counts.iloc[0])
    lines.append(f"- **Segmento con más órdenes:** **{seg_top}** ({seg_top_n} pedidos).")
else:
    lines.append("- No hay datos de **Segment** para calcular volumen por segmento.")
if not seg_profit.empty:
    seg_top_profit = seg_profit.index[0]
    lines.append(f"- **Segmento con mayor utilidad total:** **{seg_top_profit}**.")
else:
    lines.append("- No hay datos de **Profit** por **Segment** para utilidad.")

# Geografía
lines += ["", "<a id='ins-geo'></a>", "### 3) Geografía"]
if not reg_profit.empty:
    lines.append(f"- **Región más rentable (profit total):** **{reg_profit.index[0]}**.")
    # regiones con profit negativo
    neg_regs = reg_profit[reg_profit < 0]
    if len(neg_regs):
        regs_list = ", ".join(list(neg_regs.index[:5]))
        lines.append(f"- **Regiones en alerta (profit negativo):** {regs_list}.")
if not reg_margin.empty:
    reg_margin_sorted = reg_margin.sort_values(ascending=False)
    if not reg_margin_sorted.isna().all():
        best_margin_reg = reg_margin_sorted.dropna().index[0]
        lines.append(f"- **Mejor margen (Profit/Sales):** **{best_margin_reg}**.")
if not state_profit.empty:
    lines.append(f"- **Estados top por utilidad:** {', '.join(list(state_profit.head(3).index))}.")
    worst_states = state_profit[state_profit < 0]
    if len(worst_states):
        lines.append(f"- **Estados con pérdida:** {', '.join(list(worst_states.head(4).index))}.")

if reg_profit.empty and state_profit.empty:
    lines.append("- No hay datos geográficos suficientes (Region/State).")

# Categorías
lines += ["", "<a id='ins-cat'></a>", "### 4) Categorías & Productos"]
if not cat_sales.empty:
    lines.append(f"- **Categoría con mayor participación en ventas:** **{cat_sales.index[0]}**.")
if not cat_profit.empty:
    lines.append(f"- **Categoría con mayor utilidad total:** **{cat_profit.index[0]}**.")
if not sub_profit.empty:
    lines.append(f"- **Top subcategorías por utilidad:** {', '.join(list(sub_profit.head(3).index))}.")
if cat_sales.empty and cat_profit.empty and sub_profit.empty:
    lines.append("- No hay datos de categorías/subcategorías suficientes.")

# Envíos
lines += ["", "<a id='ins-ship'></a>", "### 5) Logística & Envíos"]
if not ship_counts.empty:
    mode_top, mode_top_n = ship_counts.index[0], int(ship_counts.iloc[0])
    lines.append(f"- **Modo de envío más usado:** **{mode_top}** ({mode_top_n} pedidos).")
if not ship_profit.empty:
    lines.append(f"- **Modo de envío más rentable:** **{ship_profit.index[0]}**.")
    neg_modes = ship_profit[ship_profit < 0]
    if len(neg_modes):
        lines.append(f"- **Modos de envío con pérdida:** {', '.join(list(neg_modes.index))}.")
if ship_counts.empty and ship_profit.empty:
    lines.append("- No hay datos de **Ship_Mode** suficientes.")

# Rentabilidad global
lines += ["", "<a id='ins-rentab'></a>", "### 6) Rentabilidad global"]
if profitable_rate is not None:
    lines.append(f"- **Órdenes rentables:** **{profitable_rate:.1%}** de las órdenes (Profit > 0).")
if global_margin is not None and not np.isnan(global_margin):
    lines.append(f"- **Margen global (Profit/Sales):** **{global_margin:.2%}**.")
if profitable_rate is None and (global_margin is None or np.isnan(global_margin)):
    lines.append("- No se pudo calcular tasa de rentabilidad ni margen global por ausencia de Sales/Profit.")

# Próximos pasos
lines += [
    "", "<a id='ins-next'></a>", "### 7) Próximos pasos",
    "- **Precios/Descuentos por región**: ajustar en regiones o estados con pérdida.",
    "- **Estrategia de envío**: restringir modalidades con pérdida o aplicar fee mínimo.",
    "- **Portafolio**: impulsar categorías con mayor utilidad; revisar SKUs con pérdidas.",
    "- **Segmentación**: campañas dedicadas al segmento con **mejor utilidad**.",
]

md = "\n".join(lines)
display(Markdown(md))


# 📌 Insights & Conclusiones — Superstore
- [1) Calidad de datos](#ins-calidad)
- [2) Clientes & Segmentos](#ins-segmentos)
- [3) Geografía](#ins-geo)
- [4) Categorías & Productos](#ins-cat)
- [5) Logística & Envíos](#ins-ship)
- [6) Rentabilidad global](#ins-rentab)
- [7) Próximos pasos](#ins-next)

<a id='ins-calidad'></a>
### 1) Calidad de datos
- **Ship_Date con nulos (antes):** **106** filas (del reporte de nulos previo).
- **Ship_Date con nulos (actual):** **0** filas.
- La regla **Ship_Date = Order_Date + 5 días** se aplicó donde hubo *Order_Date* disponible.

<a id='ins-segmentos'></a>
### 2) Clientes & Segmentos
- **Segmento con más órdenes:** **Consumer** (5191 pedidos).
- **Segmento con mayor utilidad total:** **Consumer**.

<a id='ins-geo'></a>
### 3) Geografía
- **Región más rentable (profit total):** **West**.
- **Mejor margen (Profit/Sales):** **West**.
- **Estados top por utilidad:** California, New York, Washington.
- **Estados con pérdida:** Oregon, Florida, Arizona, Tennessee.

<a id='ins-cat'></a>
### 4) Categorías & Productos
- **Categoría con mayor participación en ventas:** **Technology**.
- **Categoría con mayor utilidad total:** **Technology**.
- **Top subcategorías por utilidad:** Copiers, Phones, Accessories.

<a id='ins-ship'></a>
### 5) Logística & Envíos
- **Modo de envío más usado:** **Standard Class** (5968 pedidos).
- **Modo de envío más rentable:** **Standard Class**.

<a id='ins-rentab'></a>
### 6) Rentabilidad global
- **Órdenes rentables:** **80.6%** de las órdenes (Profit > 0).
- **Margen global (Profit/Sales):** **12.47%**.

<a id='ins-next'></a>
### 7) Próximos pasos
- **Precios/Descuentos por región**: ajustar en regiones o estados con pérdida.
- **Estrategia de envío**: restringir modalidades con pérdida o aplicar fee mínimo.
- **Portafolio**: impulsar categorías con mayor utilidad; revisar SKUs con pérdidas.
- **Segmentación**: campañas dedicadas al segmento con **mejor utilidad**.

In [4]:
# %% [markdown]
# ## 🎨 Informe con estilo — Insights & Conclusiones (Superstore)
# Pega esta celda **debajo** de donde ya tienes `df` construido.
# Renderiza un informe con **bookmarks**, **KPIs** y secciones con mejor estilo visual.
# - Mantiene el conteo de nulos previo de `Ship_Date` (desde `/content/eda_outputs/missing_report.csv` si existe).
# - No modifica `df`; solo calcula y muestra hallazgos.

# %%
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import HTML, Markdown, display

# =========================
# 1) Estilos (CSS + encabezado)
# =========================
display(HTML("""
<style>
:root{
  --bg:#0b1324; --card:#121a2b; --muted:#9fb2c8; --text:#e8eef6;
  --accent:#6ee7b7; --accent2:#60a5fa; --warn:#f59e0b; --danger:#fb7185;
}
.report-wrap{font-family:Inter,system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;color:var(--text);background:linear-gradient(180deg,#0b1324,#0b1324);padding:18px 16px;border-radius:16px;border:1px solid #1e2a41;}
.report-title{display:flex;align-items:center;gap:10px;margin:0 0 8px 0;font-weight:800;font-size:22px;letter-spacing:.2px}
.pill{display:inline-block;padding:4px 10px;border-radius:999px;background:rgba(110,231,183,.15);color:var(--accent);border:1px solid rgba(110,231,183,.35);font-size:12px;margin-right:6px}
.nav{display:flex;flex-wrap:wrap;gap:8px;margin:6px 0 14px 0}
.nav a{font-size:12px;text-decoration:none;color:var(--muted);border:1px solid #2a3a59;border-radius:999px;padding:6px 10px}
.nav a:hover{color:var(--text);border-color:#3a4c73}
.section{background:var(--card);border:1px solid #1e2a41;border-radius:14px;padding:14px;margin:10px 0 16px 0}
.section h3{margin:0 0 10px 0;font-size:16px;letter-spacing:.2px}
.hr{height:1px;background:#1e2a41;margin:10px 0 14px 0}
.kpis{display:grid;grid-template-columns:repeat(auto-fit,minmax(160px,1fr));gap:10px;margin:6px 0 4px 0}
.kpi{background:linear-gradient(180deg,#0f1a2e 0%, #0c1526 100%);border:1px solid #213150;border-radius:12px;padding:10px}
.kpi .label{color:var(--muted);font-size:11px}
.kpi .value{font-weight:800;font-size:18px;letter-spacing:.3px;margin-top:2px}
.list{margin:0;padding-left:16px}
.bad{color:var(--danger);font-weight:600}
.good{color:var(--accent);font-weight:600}
.warn{color:var(--warn);font-weight:600}
.small{color:var(--muted);font-size:12px}
.code{font-family:ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace; background:#0e1729;border:1px solid #1f2c48;border-radius:8px;padding:2px 6px}
</style>
"""))

# =========================
# 2) Utilidades
# =========================
def safe_has(col):
    return isinstance(col, str) and (col in df.columns)

def read_ship_nulls_before():
    path = Path("/content/eda_outputs/missing_report.csv")
    if path.exists():
        try:
            mr = pd.read_csv(path, index_col=0)
            idx = mr.index.astype(str)
            candidates = ["Ship_Date","Ship Date","Ship_Date "]
            for key in candidates:
                if key in idx:
                    return int(mr.loc[key, "missing"])
        except Exception:
            pass
    return None

def pct(n, d):
    return f"{(100*n/d):.1f}%" if d else "0.0%"

def fmt_num(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "—"
    if isinstance(x,(int,np.integer)):
        return f"{x:,}"
    try:
        return f"{x:,.2f}"
    except Exception:
        return str(x)

# Column mapping (asumimos columnas ya normalizadas)
sales_col    = "Sales"        if safe_has("Sales") else None
profit_col   = "Profit"       if safe_has("Profit") else None
segment_col  = "Segment"      if safe_has("Segment") else None
region_col   = "Region"       if safe_has("Region") else None
state_col    = "State"        if safe_has("State") else None
cat_col      = "Category"     if safe_has("Category") else None
subcat_col   = "Sub_Category" if safe_has("Sub_Category") else None
shipmode_col = "Ship_Mode"    if safe_has("Ship_Mode") else None
order_col    = "Order_Date"   if safe_has("Order_Date") else None
ship_col     = "Ship_Date"    if safe_has("Ship_Date") else None

# =========================
# 3) Cálculos base
# =========================
rows = len(df)
ship_nulls_before = read_ship_nulls_before()
ship_nulls_after  = int(df[ship_col].isna().sum()) if ship_col else None

seg_counts = df[segment_col].value_counts(dropna=False) if segment_col else pd.Series(dtype=int)
seg_profit = df.groupby(segment_col)[profit_col].sum().sort_values(ascending=False) if (segment_col and profit_col) else pd.Series(dtype=float)

reg_profit = df.groupby(region_col)[profit_col].sum().sort_values(ascending=False) if (region_col and profit_col) else pd.Series(dtype=float)
reg_sales  = df.groupby(region_col)[sales_col].sum().sort_values(ascending=False)  if (region_col and sales_col)  else pd.Series(dtype=float)
reg_margin = (reg_profit / reg_sales).replace([np.inf, -np.inf], np.nan) if (not reg_profit.empty and not reg_sales.empty) else pd.Series(dtype=float)

state_profit = df.groupby(state_col)[profit_col].sum().sort_values(ascending=False) if (state_col and profit_col) else pd.Series(dtype=float)

cat_sales  = df.groupby(cat_col)[sales_col].sum().sort_values(ascending=False)  if (cat_col and sales_col)  else pd.Series(dtype=float)
cat_profit = df.groupby(cat_col)[profit_col].sum().sort_values(ascending=False) if (cat_col and profit_col) else pd.Series(dtype=float)
sub_profit = df.groupby(subcat_col)[profit_col].sum().sort_values(ascending=False) if (subcat_col and profit_col) else pd.Series(dtype=float)

ship_counts = df[shipmode_col].value_counts(dropna=False) if shipmode_col else pd.Series(dtype=int)
ship_profit = df.groupby(shipmode_col)[profit_col].sum().sort_values(ascending=False) if (shipmode_col and profit_col) else pd.Series(dtype=float)

profitable_rate = None
global_margin = None
if profit_col and sales_col:
    profitable_rate = (df[profit_col] > 0).mean()
    total_sales = df[sales_col].sum()
    total_profit = df[profit_col].sum()
    global_margin = (total_profit / total_sales) if total_sales else np.nan

# Strings auxiliares
seg_top_txt = f"{seg_counts.index[0]} ({int(seg_counts.iloc[0])} órdenes)" if not seg_counts.empty else "—"
seg_top_profit_txt = f"{seg_profit.index[0]}" if not seg_profit.empty else "—"
reg_top_profit_txt = f"{reg_profit.index[0]}" if not reg_profit.empty else "—"
reg_best_margin_txt = "—"
if not reg_margin.empty and not reg_margin.dropna().empty:
    reg_best_margin_txt = f"{reg_margin.dropna().sort_values(ascending=False).index[0]}"
states_top_txt = ", ".join(list(state_profit.head(3).index)) if not state_profit.empty else "—"
neg_regs = ", ".join(list(reg_profit[reg_profit < 0].index[:5])) if not reg_profit.empty and (reg_profit < 0).any() else "—"
neg_modes = ", ".join(list(ship_profit[ship_profit < 0].index)) if not ship_profit.empty and (ship_profit < 0).any() else "—"
cat_top_sales_txt = f"{cat_sales.index[0]}" if not cat_sales.empty else "—"
cat_top_profit_txt = f"{cat_profit.index[0]}" if not cat_profit.empty else "—"
sub_top_profit_txt = ", ".join(list(sub_profit.head(3).index)) if not sub_profit.empty else "—"
ship_top_mode_txt = f"{ship_counts.index[0]} ({int(ship_counts.iloc[0])})" if not ship_counts.empty else "—"
ship_top_profit_txt = f"{ship_profit.index[0]}" if not ship_profit.empty else "—"

# KPIs
kpi_rate = f"{(profitable_rate*100):.1f}%" if profitable_rate is not None else "—"
kpi_margin = f"{(global_margin*100):.2f}%" if global_margin is not None and not np.isnan(global_margin) else "—"
kpi_orders = f"{rows:,}"

# =========================
# 4) Render del informe (HTML + anchors)
# =========================
html = f"""
<div class="report-wrap">
  <div class="report-title">📊 Insights & Conclusiones — Superstore</div>
  <div class="nav">
    <a href="#ins-calidad">1) Calidad</a>
    <a href="#ins-segmentos">2) Segmentos</a>
    <a href="#ins-geo">3) Geografía</a>
    <a href="#ins-cat">4) Categorías</a>
    <a href="#ins-ship">5) Envíos</a>
    <a href="#ins-rentab">6) Rentabilidad</a>
    <a href="#ins-next">7) Próximos pasos</a>
  </div>

  <div class="kpis">
    <div class="kpi"><div class="label">Órdenes</div><div class="value">{kpi_orders}</div></div>
    <div class="kpi"><div class="label">Órdenes rentables</div><div class="value">{kpi_rate}</div></div>
    <div class="kpi"><div class="label">Margen global (Profit/Sales)</div><div class="value">{kpi_margin}</div></div>
  </div>

  <div id="ins-calidad" class="section">
    <h3>1) Calidad de datos</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Ship_Date nulos <span class="small">(reporte previo)</span>: <b>{'—' if ship_nulls_before is None else ship_nulls_before}</b></li>
      <li>Ship_Date nulos <span class="small">(estado actual)</span>: <b>{'—' if ship_nulls_after is None else ship_nulls_after}</b></li>
      <li>Regla aplicada: <code class="code">Ship_Date = Order_Date + 5 días</code> (cuando <code class="code">Order_Date</code> existe).</li>
    </ul>
  </div>

  <div id="ins-segmentos" class="section">
    <h3>2) Clientes & Segmentos</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Segmento con más órdenes: <b class="good">{seg_top_txt}</b></li>
      <li>Segmento con mayor utilidad total: <b>{seg_top_profit_txt}</b></li>
    </ul>
  </div>

  <div id="ins-geo" class="section">
    <h3>3) Geografía</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Región más rentable (profit total): <b class="good">{reg_top_profit_txt}</b></li>
      <li>Mejor margen (Profit/Sales): <b>{reg_best_margin_txt}</b></li>
      <li>Estados top por utilidad: <b>{states_top_txt}</b></li>
      <li>Regiones con pérdida: <b class="bad">{neg_regs}</b></li>
    </ul>
  </div>

  <div id="ins-cat" class="section">
    <h3>4) Categorías & Productos</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Categoría con mayor participación en ventas: <b>{cat_top_sales_txt}</b></li>
      <li>Categoría con mayor utilidad total: <b class="good">{cat_top_profit_txt}</b></li>
      <li>Top subcategorías por utilidad: <b>{sub_top_profit_txt}</b></li>
    </ul>
  </div>

  <div id="ins-ship" class="section">
    <h3>5) Logística & Envíos</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Modo de envío más usado: <b>{ship_top_mode_txt}</b></li>
      <li>Modo de envío más rentable: <b>{ship_top_profit_txt}</b></li>
      <li>Modos de envío con pérdida: <b class="bad">{neg_modes}</b></li>
    </ul>
  </div>

  <div id="ins-rentab" class="section">
    <h3>6) Rentabilidad global</h3>
    <div class="hr"></div>
    <ul class="list">
      <li>Tasa de órdenes rentables: <b class="good">{kpi_rate}</b></li>
      <li>Margen global: <b>{kpi_margin}</b></li>
    </ul>
  </div>

  <div id="ins-next" class="section">
    <h3>7) Próximos pasos</h3>
    <div class="hr"></div>
    <ul class="list">
      <li><b>Precios/Descuentos por región</b>: ajustar en regiones/estados con pérdida.</li>
      <li><b>Estrategia de envío</b>: restringir modalidades con pérdida o agregar fee mínimo.</li>
      <li><b>Portafolio</b>: impulsar categorías/top subcategorías; revisar SKUs de Furniture con baja contribución.</li>
      <li><b>Segmentación</b>: campañas dedicadas al segmento con mejor utilidad (cross-sell).</li>
    </ul>
  </div>
</div>
"""

)
display(HTML(html))


SyntaxError: unmatched ')' (ipython-input-1403497781.py, line 236)